In [1]:
import json
from tqdm import tqdm

In [8]:
big = "memoryalpha"
small = "stexpanded"
top_k = 1

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/_matched_communities_leiden/top_10_pairs/" + small + "-" + big + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_big_file = "./results/communities_leiden/" + big + "/level_0.txt"
communities_small_file = "./results/communities_leiden/" + small + "/level_0.txt"

small_communities = []
big_communities = []
gold_pairs = []
matched_communities = []

In [9]:
with (open(communities_big_file) as cbf, open(communities_small_file) as csf,
      open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [10]:
matched_communities

[[0, [0]],
 [1, [0]],
 [2, [6]],
 [3, [0]],
 [4, [6]],
 [5, [6]],
 [6, [10]],
 [7, [0]],
 [8, [0]],
 [9, [4]],
 [10, [6]],
 [11, [6]],
 [12, [0]],
 [13, [1]],
 [14, [0]],
 [15, [4]],
 [16, [0]],
 [17, [12]],
 [18, [5]],
 [19, [6]],
 [20, [4]],
 [21, [4]],
 [22, [0]],
 [23, [6]],
 [24, [5]],
 [25, [16]]]

In [11]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [12]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        community_1 = small_communities[matched_community[0]]
        community_2 = set()
        for i in range(0, top_k):
            community_2.update(big_communities[matched_community[1][i]])
        if gold_pair[1] in community_1 and gold_pair[0] in community_2:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:00<00:00, 446.20it/s]

Gold pairs in matched communities top 1: 47.2972972972973%


In [13]:
found_big = 0
found_small = 0
for gold_pair in gold_not_exact:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")
print("Small: " + str(found_small / all_pairs * 100) + "%")

Big: 100.0%
Small: 100.0%
